In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/72.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [2]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 136776192


In [3]:
model[0].max_seq_length = 512
model[0].do_lower_case = True

In [4]:
!kaggle datasets download coldn00ldes/sarcasm-corpus-v2oraby-et-al
!unzip sarcasm-corpus-v2oraby-et-al.zip

Dataset URL: https://www.kaggle.com/datasets/coldn00ldes/sarcasm-corpus-v2oraby-et-al
License(s): Attribution 4.0 International (CC BY 4.0)
 96% 1.00M/1.04M [00:01<00:00, 1.01MB/s]
100% 1.04M/1.04M [00:01<00:00, 1.04MB/s]
Archive:  sarcasm-corpus-v2oraby-et-al.zip
  inflating: GEN-sarc-notsarc.csv    
  inflating: HYP-sarc-notsarc.csv    
  inflating: RQ-sarc-notsarc.csv     


In [5]:
import pandas as pd

df = pd.read_csv("GEN-sarc-notsarc.csv")

class_mapping = {'notsarc': 0, 'sarc': 1}
df['class_encoded'] = df['class'].map(class_mapping)

In [6]:
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(df, test_size=0.2,  stratify=df['class_encoded'], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['class_encoded'], random_state=42)

In [7]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

CUDA Available: True
Device Name: Tesla T4


In [8]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW

from sklearn.metrics import precision_recall_fscore_support
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, InputExample

import os
import zipfile

from transformers import AutoTokenizer, AutoModel
from peft import PeftModel, PrefixTuningConfig
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
def prepare_data(df, text_column, label_column):
    lst = []
    for _, row in df.iterrows():
        lst.append(InputExample(texts=[row[text_column]], label=row[label_column]))
    return lst

train_examples = prepare_data(train_data, text_column="text", label_column="class_encoded")
validation_examples = prepare_data(val_data, text_column="text", label_column="class_encoded")

In [10]:
def collate_fn(batch):
    texts = [example.texts[0] for example in batch]
    labels = [example.label for example in batch]

    tokenized = model.tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    labels = torch.tensor(labels, dtype=torch.float)

    return tokenized, labels

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_examples, shuffle=False, batch_size=16, collate_fn=collate_fn)

In [11]:
base_transformer = model._first_module().auto_model

prefix_config = PrefixTuningConfig(
    task_type=None,
    num_virtual_tokens=20,
    encoder_hidden_size=base_transformer.config.hidden_size,
)

peft_model = PeftModel(base_transformer, prefix_config)
model._first_module().auto_model = peft_model

In [12]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 368640


In [13]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: PeftModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [14]:
class ClassificationModel(nn.Module):
    def __init__(self, base_model):
        super(ClassificationModel, self).__init__()
        self.base_model = base_model
        self.classifier = nn.Linear(base_model.get_sentence_embedding_dimension(), 1)

    def forward(self, tokenized_inputs):
        embeddings = self.base_model(tokenized_inputs)["sentence_embedding"]
        logits = self.classifier(embeddings)
        return logits

In [15]:
classification_model = ClassificationModel(model).to(device)

In [16]:
trainable_params = filter(lambda p: p.requires_grad, classification_model.parameters())
optimizer = AdamW(trainable_params, lr=0.0001)
loss_fn = nn.BCEWithLogitsLoss()

In [17]:
metrics_per_epoch = []
num_epochs = 20
checkpoint_dir = "model_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(num_epochs):
    classification_model.train()
    all_train_predictions, all_train_labels = [], []
    train_loss = 0.0
    val_loss = 0.0

    for batch in train_dataloader:
        tokenized_inputs, labels = batch
        tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = classification_model(tokenized_inputs)
        loss = loss_fn(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        probabilities = torch.sigmoid(logits).squeeze()
        predictions = (probabilities > 0.5).long()

        all_train_predictions.extend(predictions.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())

    train_loss /= len(train_dataloader)
    train_precision, train_recall, train_f1, train_support = precision_recall_fscore_support(
        all_train_labels, all_train_predictions, average=None, zero_division = 0
    )

    # Validation phase
    classification_model.eval()
    all_val_predictions, all_val_labels = [], []
    with torch.no_grad():
        for batch in validation_dataloader:
            tokenized_inputs, labels = batch
            tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}
            labels = labels.to(device)

            logits = classification_model(tokenized_inputs)
            loss = loss_fn(logits.squeeze(), labels)
            val_loss += loss.item()

            probabilities = torch.sigmoid(logits).squeeze()
            predictions = (probabilities > 0.5).long()

            all_val_predictions.extend(predictions.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    val_loss /= len(validation_dataloader)
    val_precision, val_recall, val_f1, val_support = precision_recall_fscore_support(
        all_val_labels, all_val_predictions, average=None, zero_division=0
    )

    # Save metrics for this epoch
    epoch_metrics = {
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_precision": train_precision,
        "train_recall": train_recall,
        "train_f1": train_f1,
        "val_precision": val_precision,
        "val_recall": val_recall,
        "val_f1": val_f1
    }
    metrics_per_epoch.append(epoch_metrics)

    base_model_path = os.path.join(checkpoint_dir, f"epoch{epoch + 1}_gist_model.pth")
    torch.save(classification_model.base_model.state_dict(), base_model_path)

    classifier_path = os.path.join(checkpoint_dir, f"epoch{epoch + 1}_classifier_weights.pth")
    torch.save(classification_model.classifier.state_dict(), classifier_path)

    model_path = os.path.join(checkpoint_dir, f"epoch{epoch + 1}_classification_model.pth")
    torch.save(classification_model.state_dict(), model_path)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training -> Loss: {train_loss}")
    print(f"  Precision: {train_precision}    Recall: {train_recall}    F1: {train_f1}")
    print(f"Validation -> Loss: {val_loss}")
    print(f"  Precision: {val_precision}    Recall: {val_recall}    F1: {val_f1}")
    print("============================================================================================================")

Epoch 1/20
Training -> Loss: 0.6258811300700428
  Precision: [0.6404533  0.67751736]    Recall: [0.71510736 0.59854294]    F1: [0.67572464 0.63558632]
Validation -> Loss: 0.5800061305848564
  Precision: [0.68055556 0.72260274]    Recall: [0.75153374 0.64723926]    F1: [0.71428571 0.6828479 ]
Epoch 2/20
Training -> Loss: 0.5454230111069475
  Precision: [0.72417582 0.7461786 ]    Recall: [0.75805215 0.71127301]    F1: [0.74072686 0.72830781]
Validation -> Loss: 0.552998968013903
  Precision: [0.70996979 0.7165109 ]    Recall: [0.7208589  0.70552147]    F1: [0.71537291 0.71097372]
Epoch 3/20
Training -> Loss: 0.5163987628155691
  Precision: [0.74839683 0.75672515]    Recall: [0.7607362  0.74424847]    F1: [0.75451607 0.75043495]
Validation -> Loss: 0.5465139826623405
  Precision: [0.69187675 0.73220339]    Recall: [0.75766871 0.66257669]    F1: [0.72327965 0.69565217]
Epoch 4/20
Training -> Loss: 0.5054906641413097
  Precision: [0.75046624 0.76489152]    Recall: [0.77147239 0.7434816 ]   

In [18]:
metrics_df = pd.DataFrame(metrics_per_epoch)
metrics_df.to_csv("epoch_metrics.csv", index=False)